In [12]:
workspace()
using Bolas
using Derivatives


    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:210
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:215.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:215
is ambiguous with: 
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:210.
To fix, define 
    +(Bola,Bola)
before the new definition.
    -(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:225
is ambiguous with: 
    -(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:230.
To fix, define 
    -(Bola,Bola)
before the new definition.
    -(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:230
is ambiguous with: 
    -(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:225.
To fix, define 
    -(Bola,Bola)
before the new definition.
    ^(Derive,Real) at /home/pablo1/Documents

In [13]:
A=Bola(0,1)
B=Bola(1.2,1)

interseccion_de_bolas(B,A)

Bola(0.6,0.4)

In [24]:
function quitar_no_deseadas(F::Function,B,x)
    
    temp=Bola[]
    dF(x)=F(makex(x)).d
    
    for i=1:length(B)
        if contiene(F(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
    
    
end

function operador_de_newton_bola(F::Function,B::Bola)
    
    dF(x)=F(makex(x)).d
    m=Bola(B.centro,0)
    
    return (m-F(m)/(dF(B)))
    
end   

function metodo_newton_bola(F::Function, B::Bola)
    
    X=bisectar_bolas([B])
    
    i=0
    tolbis=1e-3
    tolradio=1e-5
    
    while(X[1].radio>tolbis)
        
        n=length(X)
        
        dF(x)=F(makex(x))

        X=bisectar_bolas(X)
        X=quitar_no_deseadas(F,X,0)

        
        i+=1
        @show i
 
    end
    
    while(tolradio<X[1].radio)
        
        for i=1:length(X)
            X[i]=interseccion_de_bolas(B,operador_de_newton_bola(F,X[i]))
        end
    end
    
    tolsimilar = 1e-2
    
    
    return(X)
    
end


f(x)=(x-1)*(x-2)*(x-3)*(x-4)*(x-5)
metodo_newton_bola(f,Bola(BigFloat(2),BigFloat(10)))

i => 1
i => 2
i => 3
i => 4
i => 5
i => 6
i => 7
i => 8
i => 9
i => 10
i => 11
i => 12
i => 13


6-element Array{Bola,1}:
 Bola(2e+00 with 256 bits of precision,0e+00 with 256 bits of precision)                                                                                                                                                              
 Bola(1.999999997664350821622093055792080981676611382669058738736345091709838995550691e+00 with 256 bits of precision,5.148121385738302036382040596937644323401743857785310688778581873030593590355652e-33 with 256 bits of precision)
 Bola(2.000000002320964667663534151017415868427663251804391341191686175379449430995466e+00 with 256 bits of precision,4.89282716055259312168572161169287793603429287449853340978209614870585768602744e-33 with 256 bits of precision) 
 Bola(3.000000000187499774743721182528282661978339109513780602681245949411271423195172e+00 with 256 bits of precision,1.379638996535498044770646635886886683896380482464596888509981851900307182792242e-41 with 256 bits of precision)
 Bola(4e+00 with 256 bits of precision,0e+00 with 2

In [104]:
distancia(Bola(3.000002260893497,1.4110593392949528e-15),Bola(2.999997739770821,1.4098746056536714e-15))
f(2.999999748864685966332584026743665184890229144155282698741380181645717617816465)

-5.022704385489414e-7

In [110]:
contiene(Bola(1.0000000018626451,1.862645149230957e-9),1)

true

In [86]:
operador_de_newton_bola(f,Bola(3.0056896291766835,0.5880009079680009))

LoadError: No se puede dividir por una bola que contine el 0
while loading In[86], in expression starting on line 1

In [110]:
f(B)

Bola(2.3201169227365472,3.3201169227365472)

In [51]:
contiene(dF(B).d,0)

false

In [24]:
2^10

1024

In [120]:
function quitar_bolas_similares(B,tol::Real)
   
    temp=Bola[]
    n=length(B)
    
    
    
    
    
end

X[i] = operador_de_newton_bola(F,X[i]) => Bola(0.7231301601484298,7.8019239264248)
X[i] = operador_de_newton_bola(F,X[i]) => Bola(1.5820849986238987,68.11533257093657)


0-element Array{Bola,1}

In [77]:
interseccion_de_bolas(Bola(0.7999999999999999,0.6),Bola(1.7999999999999998,0.3999999999999999))

Bola(1.4,0.0)

In [102]:
f(1.2)

1.24